In [2]:
import pennylane as qml
from pennylane import numpy as np

In [5]:
dev = qml.device("default.qubit.autograd", wires=1)

In [22]:
def decomposition(params):
    qml.PhaseShift(params[0], wires=0)
    #qml.RX(params[1], wires=0)
    #qml.PhaseShift(params[2], wires=0)
    qml.Rot(params[1],params[2],params[3],wires=0)
    

# notice to use diff_method="backprop"


@qml.qnode(dev, diff_method="backprop")

def f(params, state, apply_inv=True):
    qml.BasisState(np.array(state), wires=0)
    decomposition(params)
    if apply_inv:
        qml.PauliX(wires=0).inv()
    return qml.state()

In [23]:
params = np.random.random(4)
Id = np.eye(2, requires_grad=False)
state_vector=np.array([[0],[1]])
print(state_vector[0])

[0]


In [24]:
def cost_state(params, i):
    state=state_vector[i]
    return np.sum(np.abs(f(params, state) - Id[i]))

def cost(params):
    return sum(cost_state(params, i) for i in range(2))

In [25]:
opt = qml.GradientDescentOptimizer(stepsize=0.01)

for i in range(3000+1):
    params = opt.step(cost, params)

    if i % 300 == 0:
        print(f"Cost at step {i}:", cost(params))

Cost at step 0: 3.850552311240353
Cost at step 300: 1.473377408991543
Cost at step 600: 0.023257203156229
Cost at step 900: 0.023290674684158165
Cost at step 1200: 0.023324581157765433
Cost at step 1500: 0.023358928465624058
Cost at step 1800: 0.023393722576656752
Cost at step 2100: 0.02342896954123189
Cost at step 2400: 0.023464675492241766
Cost at step 2700: 0.02350084664621641
Cost at step 3000: 0.02353748930447359


In [113]:
opt = qml.GradientDescentOptimizer(stepsize=0.01)

for i in range(3000+1):
    params = opt.step(cost, params)

    if i % 300 == 0:
        print(f"Cost at step {i}:", cost(params))

Cost at step 0: 2.456014282759185
Cost at step 300: 1.6440519781546081
Cost at step 600: 0.03099321552951417
Cost at step 900: 0.031113818644278455
Cost at step 1200: 0.031235795177878497
Cost at step 1500: 0.03135916225995361
Cost at step 1800: 0.031483937224800265
Cost at step 2100: 0.03161013761377303
Cost at step 2400: 0.03173778117778315
Cost at step 2700: 0.03186688587978385
Cost at step 3000: 0.03199746989728698


In [20]:
def get_unitary(params):
    return np.array([f(params, state, apply_inv=False) for state in state_vector])

In [26]:
print("\nOptimized unitary:\n", (get_unitary(params)))
print("Ideal unitary:\n", qml.PauliX.matrix)

print("\nOptimized params:", params.numpy())
print("Expected params: ", np.array([np.pi / 2] * 3))


Optimized unitary:
 [[-0.00282203+0.00284476j  0.99997294-0.0061702j ]
 [ 0.99994823+0.00935296j  0.00281296+0.00285373j]]
Ideal unitary:
 [[0 1]
 [1 0]]

Optimized params: [3.14477554 0.79557962 3.1496068  0.78323904]
Expected params:  [1.57079633 1.57079633 1.57079633]


In [15]:
print(Id[1])

[0. 1.]


In [17]:
f([.9,1,5,1.3],0)

tensor([0.70710678+0.j, 0.70710678+0.j], requires_grad=True)